#  🔍 Improve retrieval by embedding meaningful metadata 🏷️

<a target="_blank" href="https://colab.research.google.com/github/anakin87/notebooks/blob/main/improve_retrieval_by_embedding_metadata.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="200"/>
</a>

In this notebook, I do some experiments on embedding meaningful metadata to improve Document retrieval.

I use the [Haystack LLM orchestration framework](https://github.com/deepset-ai/haystack), which provides several out-of-the box features for embedding creation and retrieval. In particular, I use the 2.0-beta version.

In [1]:
%%capture
! pip install wikipedia haystack-ai sentence_transformers rich

In [2]:
import rich

## Load data from Wikipedia

We are going to download the Wikipedia pages related to some bands, using the python library `wikipedia`.

These pages are converted into Haystack Documents.

In [3]:
some_bands="""The Beatles
Rolling stones
Dire Straits
The Cure
The Smiths""".split("\n")

In [4]:
import wikipedia
from haystack.preview.dataclasses import Document

raw_docs=[]

for title in some_bands:
    page = wikipedia.page(title=title, auto_suggest=False)
    doc = Document(content=page.content, meta={"title": page.title, "url":page.url})
    raw_docs.append(doc)

## 🔧 Setup the experiment

### Utility functions to create Pipelines

The **indexing Pipeline** transforms the Documents and stores them (with vectors) in a Document Store. The **retrieval Pipeline** takes a query as input and perform the vector search.


I build some utility functions to create different indexing and retrieval Pipelines.

In fact, I am interested in comparing the standard approach (where we only embed text) with the embedding metadata strategy (we embed text + meaningful metadata).



In [5]:
from haystack.preview import Pipeline
from haystack.preview.document_stores import InMemoryDocumentStore
from haystack.preview.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.preview.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.preview.components.writers import DocumentWriter
from haystack.preview.components.writers.document_writer import DuplicatePolicy
from haystack.preview.components.retrievers import InMemoryEmbeddingRetriever

In [6]:
def create_indexing_pipeline(document_store, metadata_fields_to_embed):

  indexing = Pipeline()
  indexing.add_component("cleaner", DocumentCleaner())
  indexing.add_component("splitter", DocumentSplitter(split_by='sentence', split_length=2))

  # in the following componente, we can specify the parameter `metadata_fields_to_embed`, with the metadata to embed
  indexing.add_component("doc_embedder", SentenceTransformersDocumentEmbedder(model_name_or_path="thenlper/gte-large", device="cuda:0", metadata_fields_to_embed=metadata_fields_to_embed)
  )
  indexing.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE))

  indexing.connect("cleaner", "splitter")
  indexing.connect("splitter", "doc_embedder")
  indexing.connect("doc_embedder", "writer")

  return indexing


In [7]:
def create_retrieval_pipeline(document_store):

  retrieval = Pipeline()
  retrieval.add_component("text_embedder", SentenceTransformersTextEmbedder(model_name_or_path="thenlper/gte-large", device="cuda:0"))
  retrieval.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, scale_score=False, top_k=3))

  retrieval.connect("text_embedder", "retriever")

  return retrieval

###  Create the Pipelines

Let's define 2 Document Stores, to compare the different approaches.

In [8]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_store_w_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")

Now, I create the 2 indexing pipelines and run them.

In [9]:
indexing_pipe_std = create_indexing_pipeline(document_store=document_store, metadata_fields_to_embed=[])

# here we specify the fields to embed
# we select the field `title`, containing the name of the band
indexing_pipe_w_embedded_metadata = create_indexing_pipeline(document_store=document_store_w_embedded_metadata, metadata_fields_to_embed=["title"])

In [10]:
indexing_pipe_std.run({"cleaner":{"documents":raw_docs}})
indexing_pipe_w_embedded_metadata.run({"cleaner":{"documents":raw_docs}})

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

{'writer': {'documents_written': 1233}}

In [11]:
print(len(document_store.filter_documents()))
print(len(document_store_w_embedded_metadata.filter_documents()))

1203
1203


Create the 2 retrieval pipelines.

In [12]:
retrieval_pipe_std = create_retrieval_pipeline(document_store=document_store)

retrieval_pipe_w_embedded_metadata = create_retrieval_pipeline(document_store=document_store_w_embedded_metadata)

## 🧪 Run the experiment!

In [13]:
# standard approach (no metadata embedding)

res=retrieval_pipe_std.run({"text_embedder":{"text":"have the beatles ever been to bangor?"}})
for doc in res['retriever']['documents']:
  rich.print(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document(
    id='e43f01744e901f52fe2ca20977bc36d108b9daf70064c9fe8940973dd90b9b8b',
    content='After the Beatles completed their second Hamburg residency, they enjoyed increasing popularity in 
Liverpool with the growing Merseybeat movement. However, they were growing tired of the monotony of numerous 
appearances at the same clubs night after night.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8645878610261432
)

Document(
    id='8ceaac87142dbab86ddfa8699ff5fe0db223639678796d337209e6ba45d6f30c',
    content=" The band flew to Florida, where they appeared on The Ed Sullivan Show a second time, again before 70 
million viewers, before returning to the UK on 22 February.The Beatles' first visit to the US took place when the 
nation was still mourning the assassination of President John F.",
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8637015907150832
)

Document(
    id='3670a59e1a13dda95ad11d901e5ed3639ac0b96466a11d9d03348a62f2160c7b',
    content='During the 1964 US tour, the group were confronted with racial segregation in the country at the time.
When informed that the venue for their 11 September concert, the Gator Bowl in Jacksonville, Florida, was 
segregated, the Beatles said they would refuse to perform unless the audience was integrated.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8558953464359195
)

❌ the retrieved Documents seem irrelevant

In [14]:
# embedding meaningful metadata

res=retrieval_pipe_w_embedded_metadata.run({"text_embedder":{"text":"have the beatles ever been to bangor?"}})
for doc in res['retriever']['documents']:
  rich.print(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document(
    id='1ccbbe150351e6a4921a94d530d6d4b6e6498d4b87979a6218102dbc43dcaba3',
    content='On 24 August, the group were introduced to Maharishi Mahesh Yogi in London. The next day, they 
travelled to Bangor for his Transcendental Meditation retreat.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8897091248192761
)

Document(
    id='394306bab975cfd097125ddbe469fb6addaec5396a778f97d7f0c34eb423448c',
    content='" City officials relented and agreed to allow an integrated show. The group also cancelled their 
reservations at the whites-only Hotel George Washington in Jacksonville.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8637997942650615
)

Document(
    id='8ceaac87142dbab86ddfa8699ff5fe0db223639678796d337209e6ba45d6f30c',
    content=" The band flew to Florida, where they appeared on The Ed Sullivan Show a second time, again before 70 
million viewers, before returning to the UK on 22 February.The Beatles' first visit to the US took place when the 
nation was still mourning the assassination of President John F.",
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Beatles',
        'url': 'https://en.wikipedia.org/wiki/The_Beatles',
        'source_id': '1d78308a2528ae83e8558da061f2832e9b29dd01d78aa01ab37652c899864a7a'
    },
    score=0.8637313897829038
)

✅ the first Document is relevant

In [15]:
# standard approach (no metadata embedding)

res=retrieval_pipe_std.run({"text_embedder":{"text":"What announcements did the band The Cure make in 2022?"}})
for doc in res['retriever']['documents']:
  rich.print(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document(
    id='5b7760fa7cf4a95f8f4b8f8b7aa8964cb86aff74895ccee74f9a0dc229ff86d7',
    content='" However, the year passed with no new studio release, as did the following three years.\nIn 2019, the
Cure embarked on a 23-date summer tour, consisting mostly of festival performances along with four dates in Sydney,
Australia.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.9031490171056397
)

Document(
    id='17ffb70cdbc6c5ab1d4d9d9a241878f2e358e5e63ce4506dd6e7fdf1c9013cda',
    content=' The band released their twelfth album, The Cure, on Geffen in 2004. It made a top ten debut on both 
sides of the Atlantic in July 2004.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.8883580705408408
)

Document(
    id='ea56f80b82606d4edbd9b53c930ded04596d9a7700ea3975abd7e42226f6e4d8',
    content=' Later that same month, the band issued 40 Live: CURÆTION-25 + Anniversary, a Blu-ray, DVD and CD box 
set featuring their Meltdown and Hyde Park performances from 2018 in their entireties.In interviews in June 2021, 
Smith referenced the recording of two new Cure albums, saying "One of them\'s very, very doom and gloom and the 
other one isn\'t," and that the recordings have been completed, "I just have to decide who\'s going to mix them.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.8794701833602774
)

❌ the retrieved Documents seem irrelevant

In [16]:
# embedding meaningful metadata

res=retrieval_pipe_w_embedded_metadata.run({"text_embedder":{"text":"What announcements did the band The Cure make in 2022?"}})
for doc in res['retriever']['documents']:
  rich.print(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document(
    id='7492f613100a3bf94e86dec69ae12c8b50dd7aed351fde8c333053359f3c3da7',
    content=" An update was provided in May 2022, when Smith claimed that the album would be released prior to the 
band's European tour in October 2022. This, however, didn't happen, as the tour got underway with no new album 
being released, although new material was performed.",
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.9231492660243582
)

Document(
    id='cff50184b9d0eaf8cad8ff3e2737f777c09f382cc2fe799860260c2259e43b3e',
    content=" On 14 October 2021, Gallup confirmed that he was still in the band.In March 2022, Smith confirmed 
that the first of the band's two projected new albums would be titled Songs of a Lost World.",
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.9159594827971157
)

Document(
    id='5b7760fa7cf4a95f8f4b8f8b7aa8964cb86aff74895ccee74f9a0dc229ff86d7',
    content='" However, the year passed with no new studio release, as did the following three years.\nIn 2019, the
Cure embarked on a 23-date summer tour, consisting mostly of festival performances along with four dates in Sydney,
Australia.',
    dataframe=None,
    blob=None,
    mime_type='text/plain',
    meta={
        'title': 'The Cure',
        'url': 'https://en.wikipedia.org/wiki/The_Cure',
        'source_id': 'dba54afaf1396350dcb4b251b887a661d5a00cf3fedbc8e6e664928989c17472'
    },
    score=0.9077689960142526
)

✅ the first 2 Documents are relevant

## ⚠️ Notes of caution

- This technique is not a silver bullet
- It works well when the embedded metadata are meaningful and distinctive
- I would say that the embedded metadata should be meaningful from the perspective of the embedding model. For example, I don't expect embedding numbers to work well.